# 1 - Listado de variables y selección

## Entrada
* Manufacturer: Marca / fabricante del equipo
* Model Name: Nombre / número de modelo del equipo
* Category: Tipo de notebook
* Screen Size: Tamaño diagonal de la pantalla en pulgadas
* Screen Type: Resolución y tipo de panel
* CPU: Modelo de procesador
* RAM: Capacidad de memoria
* Storage: Tipo y tamaño de almacenamiento interno
* GPU: Tipo y modelo de procesador de gráficos
* OS: Sistema operativo
* Weight: Peso

## Salida
* Price: Será reducida con rangos a valores "bajo", "medio" o "alto"

## No utilizadas
* OS Version: No usada debido a su gran cantidad de nulos y presunta poca importancia

# 2 - Análisis detallado de un conjunto de variables


In [ ]:
import pandas as pd

In [ ]:
laptops_train = pd.read_csv('laptops_train.csv')
laptops_test = pd.read_csv('laptops_test.csv')

In [ ]:
print("Entries de train: "+str(len(laptops_train)))
print("Entries de test: "+str(len(laptops_test)))

### Valores nulos

In [ ]:
laptops_train.isnull().sum()

In [ ]:
laptops_test.isnull().sum()

Este dataset solo presenta valores nulos en la columna "Operating System Version", la cual se decidió no utilizar anteriormente, por lo cual no es un problema a tener en cuenta en este caso

In [ ]:
laptops_train

In [ ]:
# Renombre de las series
new_col_names = {
    "Manufacturer":"manufacturer",
    "Model Name":"model",
    "Category":"category",
    "Screen Size":"screen_size",
    "Screen":"resolution",
    "CPU":"cpu",
    "RAM":"ram",
    " Storage":"storage", # la string original tiene el espacio, sin eso no lo cambia
    "GPU":"gpu",
    "Operating System":"os",
    #"Operating System Version":"os_version",
    "Weight":"weight",
    "Price":"price"
}

laptops_train = laptops_train.rename(columns=new_col_names).drop(columns="Operating System Version")
laptops_test = laptops_test.rename(columns=new_col_names).drop(columns="Operating System Version")

In [ ]:
laptops_train

## Variable de salida

In [ ]:
# Conversión de la variable de salida de rupias a dólares, tomando como base la cotización del 28/03/2023, según la última actualización del dataset
# Se dividió por 100 el valor obtenido debido a que los precios convertidos con la cotización de referencia se encontraban fuera del rango esperado
# Cotización: 1 INR = 0.0001217 USD


cotizacion = 0.0001217

def convertir_precio(precio):
    return int(precio * cotizacion)

laptops_train["price"] = laptops_train["price"].apply(convertir_precio)
laptops_test["price"] = laptops_test["price"].apply(convertir_precio)

In [ ]:
laptops_train

In [ ]:
laptops_test

In [ ]:
import matplotlib as plt
import plotly.express as px

In [ ]:
px.histogram(laptops_train, x="price")

In [ ]:
px.histogram(laptops_test, x="price")

In [ ]:
px.box(laptops_train, x="price")

In [ ]:
px.box(laptops_test, x="price")

In [ ]:
laptops_train["price"].describe()

In [ ]:
laptops_test["price"].describe()

Se puede apreciar como la variable de salida tiene una forma de campana gaussiana con asimetría positiva. Podemos notar como tanto el set de train y de test tienen una distribución similar, lo cual es importante controlar debido a que ambas tablas fueron provistas por el creador del dataset. A su vez se puede ver como existen valores anómalos y aberrantes en los rangos superiores del dataset. En consecuencia, esto podría generar overfitting debido a los pocos datos en este rango, y optamos por quitar estos dispositivos del dataset directamente.

In [ ]:
# Descartamos los valores anómalos y aberrantes (donde precio > 3500)

descartes_train = laptops_train[laptops_train.price > 3500].index
descartes_test = laptops_test[laptops_test.price > 3500].index

laptops_train = laptops_train.drop(descartes_train)
laptops_test = laptops_test.drop(descartes_test)

print("Valores descartados:")
print("Train:"+str(len(descartes_train)))
print("Test:"+str(len(descartes_test)))

In [ ]:
# Conversión a rangos de precios
# Rangos:
# low: 0 < price <= 1000
# mid: 1000 < price <= 1500
# high: 1500 < price <= 3500

laptops_train["price_range"] = pd.cut(x=laptops_train["price"], bins=[0, 1000, 1500, 3500], labels=["low", "mid", "high"])
laptops_test["price_range"] = pd.cut(x=laptops_test["price"], bins=[0, 1000, 1500, 3500], labels=["low", "mid", "high"])

In [ ]:
# Ordenamos el dataset por la nueva columna de price_range para que los gráficos queden mejor acomodados
# (se puede hacer agregando category_orders={"price_range": ["low", "mid", "high"]} a cada gráfico, pero de esta manera aplica para todos)

laptops_train = laptops_train.sort_values(by="price_range")
laptops_test = laptops_test.sort_values(by="price_range")

In [ ]:
laptops_train

In [ ]:
#Una ves aplicado los rangos, droppeamos la columna de precio, porque no la vamos a usar
laptops_train = laptops_train.drop(columns="price")
laptops_test = laptops_test.drop(columns="price")

In [ ]:
laptops_train

In [ ]:
px.histogram(laptops_train, x = "price_range")

## Variables de entrada

### Manufacturer

In [ ]:
px.histogram(laptops_train,x="manufacturer", color="price_range")

Se puede observar como el dataset cuenta en su mayoria con 4 manufacturadores predominantes, Lenovo, Dell, HP y Asus.

### Model

In [ ]:
px.box(laptops_train,x="model", color="price_range", height=800)

Se puede notar como el precio aumenta dependiendo de que tan nuevo es el modelo del laptop. Estas variables estan correlacionadas linealmente ?

### Category

In [ ]:
px.histogram(laptops_train, x="category", color="price_range")

Se observa como la mayoria de los datos ingresados tienen como categoria "Notebook". Ademas, la categoria "Workstation" contiene solo valores donde el rango de precio es "High" lo cual influira en la decision tomada por el modelo. Se puede notar tambien como las categorias "Gaming" y "Ultrabook" cuentan con muchos valores "mid" y "high", lo que tambien influye en las decisiones finales.

### Screen size

In [ ]:
px.histogram(laptops_train, x="screen_size", color="price_range").update_xaxes(categoryorder="category ascending")

Se puede ver como la mayoria de los datos cuentan con un tamaño de pantalla de 15,6 pulgadas. Tambien se puede notar algunos datos individuales esparcidos entre los otros tamaños

### Resolution

In [ ]:
px.histogram(laptops_train, x="resolution", color="price_range", height=600)

En este grafico demuestra que existen tres tipos de resolucion predominantes, siendo "Full HD" la que contiene mas ejemplares. 
Para facilitar el aprendizaje del modelo, modificaremos los datos de esta variable...

### CPU

In [ ]:
px.histogram(laptops_train, x="cpu", color="price_range", height=700)

Se nota como la variable con mas ejemplares es la de Intel Core i5, y como el rango de precios aumenta con CPUs mas modernos

### RAM

In [ ]:
px.histogram(laptops_train, x="ram", color="price_range", category_orders={"ram": ["2GB", "4GB", "6GB", "8GB", "12GB", "16GB", "24GB", "32GB"]})

Se puede notar como al incrementar el valor de la RAM, aumenta el precio del laptop. Es una relacion lineal

### Storage

In [ ]:
px.histogram(laptops_train, x="storage", color="price_range", height=600)

Para esta variable, aplicaremos separaciones en GB de SSD y HDD. Se puede notar en el grafico como la mayoria de laptops cuentan con 256GB de SSD. Esta variable cuenta con muchos datos esparcidos.

### GPU

In [ ]:
px.histogram(laptops_train, x="gpu", color="price_range", height=600)

Las GPU mas caras resultan ser las pertenecientes a NVidia, aumentando el precio con las gpu mas modernas. La mas comun en el dataset son las de Intel HD Graphics.

### OS

In [ ]:
px.histogram(laptops_train, x="os", color="price_range")

La gran mayoria de las laptops tiene como sistema operativo a Windows. Se aprecia una discrepancia al haber dos columnas de mac OS, que sera resuelta en breve.

### Weight

In [ ]:
px.histogram(laptops_train, x="weight", color="price_range").update_xaxes(categoryorder="category ascending")

En cuanto al Peso, se puede notar como la mayoria de los equipos pesan 2.3kg, marcando su precio como "bajo".
Observando la grafica podemos ver como el precio aumenta en los Pesos mas bajos y mas altos?, manteniendose en un precio bajo en los pesos intermedios

## Listado de dudas/preguntas para el proveedor del dataset

* De que fuentes provienen estos precios? y que metodos fueron utilizados para recolectarlos?
* Porque la columna de SO Type tiene tantos valores nulos?
* Pensas que el tipo de SO afecta al precio?
* Pensas que existen otros factores que afecten el precio de una laptop? como serían el material de chasis, distribución de teclado, etc.
* Cual crees que es la variable que mas afecta al precio de una laptop?
* Omitiste alguna variable al creear el dataset?

## 3 - Hipotesis sobre los datos

* Pensamos que la variable OS influye poco sobre el rango de precio final
* Creemos que las variables mas influyentes son GPU, Storage, RAM y CPU
* La variable Category se ve afectada principalmente por el GPU? 

## Comprobacion de la hipotesis

## Creacion de nuevas variables

## 4 - Modelado